# Utility Chains Overview

# Summarizing Documents - Summarize Chains

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [2]:
# Initialize OpenAI 
llm = OpenAI(temperature=0.9)

In [3]:
# Reading the document
with open("sample-1.txt") as f:
    data = f.read()

In [4]:
# Breaking the large data into smaller chunks
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

In [6]:
# Create documents
docs = [Document(page_content=t) for t in texts]
docs # Let's look at the documents

[Document(page_content="Title: The Computer: Revolutionizing the World of Technology\n\nIntroduction:\nThe computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.\n\nThe Birth of the Computer:\nThe computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, and Grace Hopper made significant contributions to the field, laying the groundwork for the computers we know today.\n\nThe Ev

To create an instance of load_summarizer_chain, we need to provide three arguments.

Firstly, we need to pass the desired large language model that will be used to query the user input. Secondly, we specify the type of langchain chain to be used for summarizing documents.
Lastly, we can set the verbose argument to True if we want to see all the intermediate steps involved in processing the user request and generating the output.

In [7]:
chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True)
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Title: The Computer: Revolutionizing the World of Technology

Introduction:
The computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.

The Birth of the Computer:
The computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, a

' Computers have revolutionized modern life by automating repetitive tasks, streamlining operations, and connecting people across distances. Computers have enabled easier productivity, communication, learning, and entertainment, and their potential to help people and societies is limitless with responsible and ethical development and usage.'

## HTTP Requests

### LLM Requests Chain - To hit any http URL to get the response back

In [20]:
from langchain.chains import LLMRequestsChain, LLMChain

In [21]:
template_question = """
Extract the answer to the question {query} or say "not found", if the information is not available.
{requests_result}
"""

prompt = PromptTemplate(
    input_variables = ['query', 'requests_result'],
    template = template_question
)

In [22]:
llm = OpenAI()

In [23]:
chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=prompt))

In [24]:
# Prepping the question
question = "what is the capital of Bangladesh ?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q="+question.replace(" ","+")
}

In [25]:
chain(inputs=inputs)

{'query': 'what is the capital of Bangladesh ?',
 'url': 'https://www.google.com/search?q=what+is+the+capital+of+Bangladesh+?',
 'output': '\nAnswer: Dhaka'}

In [27]:
# To look at internal functioning 
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}
